In [43]:
import ast
import random
import pickle
import json
from collections import OrderedDict
import warnings
import threading
from multiprocessing.pool import ThreadPool
import concurrent.futures
from itertools import permutations, combinations

import pandas as pd
import numpy as np
from numpy.linalg.linalg import LinAlgError
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'  # https://stackoverflow.com/questions/34387893/output-matplotlib-figure-to-svg-with-text-as-text-not-curves
from matplotlib_venn import venn2, venn3, venn3_circles
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
import statsmodels.api as sm
import dateutil
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFdr
from scipy.stats import chi2
from patsy import ModelDesc, dmatrix, dmatrices

In [380]:
# colors and class-superclass maps
with open(r'..\data\metadata\color_schemes.json') as infile:
    colors = json.load(infile)

animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic =     [1076, 1082, 1101]
impaired =     [1060, 1062, 1074, 1092, 1102]
normal =       [1091, 1093]
animal_tol = {
    1076: 'diabetic', 1082: 'diabetic', 1101: 'diabetic', 1060: 'impaired', 1062: 'impaired', 
    1074: 'impaired', 1092: 'impaired', 1102: 'impaired', 1091: 'normal', 1093: 'normal'}
ap = pd.read_excel(r'..\data\metadata\animal_phenotypes.xlsx', index_col=0)
fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)

rbg_cols = fg.loc[fg['bg_type'] == 'RBG'].index
fbg_cols = fg.loc[fg['bg_type'] == 'FBG'].index
data_cols = rbg_cols.append(fbg_cols)

data = pd.read_csv(r'..\data\interim\interim_combined_metab_lipid_table.csv', index_col=0).set_index('i').T
data = data.filter(regex='RBG|FBG', axis=0)
data = data.astype('float')
feat_cols = data.columns
data = data.join(fg[['bg_type', 'bg', 'week', 'gluc_tol', 'animal', 'ogtt', 'litter']])

### ENCODING WEEK AND LITTER AS A CATEGORICAL STRING INSTEAD OF NUMERIC
data[['week', 'litter']] = data[['week', 'litter']].astype('str')  

weight_insulin = fg['animal'].apply(lambda x: ap.loc[x, ['Insulin (AUC)', 'Weightprefastweek12']])
data = data.join(weight_insulin)
data.rename({'bg_type': 'sampling', 
             'Insulin (AUC)': 'insulin', 
             'Weightprefastweek12': 'weight'}, 
            inplace=True, axis=1)
data

,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,...,l_872,sampling,bg,week,gluc_tol,animal,ogtt,litter,insulin,weight
1102_8_RBG,16.86165,22.14457,22.57698,23.80379,24.59673,23.37160,22.84790,23.54712,25.62865,22.99648,...,16.20001,RBG,299.50000,8,impaired,1102,1738.25000,3,651.93850,121.30000
1093_9_RBG,16.67132,20.96340,23.84326,24.31631,26.20137,23.33231,23.05438,23.32567,23.16177,23.24502,...,15.91576,RBG,83.00000,9,normal,1093,993.00000,2,377.16900,94.50000
1060_8_FBG,17.59473,20.19877,23.89523,24.71491,17.14899,22.65345,22.08450,21.39721,26.74455,22.06536,...,19.83680,FBG,41.00000,8,impaired,1060,1761.75000,0,294.34300,105.10000
1060_8_RBG,20.82696,21.95974,23.75479,24.23568,17.43124,23.21182,22.66763,22.99824,25.60348,22.89618,...,16.20216,RBG,225.00000,8,impaired,1060,1761.75000,0,294.34300,105.10000
1060_9_FBG,21.80972,20.52466,24.26321,24.52041,19.16921,23.20704,22.45004,21.42402,25.01842,22.53474,...,20.23428,FBG,48.00000,9,impaired,1060,1761.75000,0,294.34300,105.10000
1060_9_RBG,22.73057,21.24059,19.53213,23.86696,26.31293,23.10995,22.59567,22.95654,25.44379,22.71898,...,16.49459,RBG,83.00000,9,impaired,1060,1761.75000,0,294.34300,105.10000
1060_10_FBG,18.74811,20.30397,23.08439,24.79238,27.20916,22.79362,22.56326,21.78463,25.54833,22.66438,...,19.27962,FBG,52.50000,10,impaired,1060,1761.75000,0,294.34300,105.10000
1060_10_RBG,18.99010,22.02089,23.52826,23.99806,25.95612,23.45438,22.93540,23.57147,25.37834,23.05614,...,15.24025,RBG,206.00000,10,impaired,1060,1761.75000,0,294.34300,105.10000
1062_8_FBG,18.09824,21.01905,24.87335,24.78738,21.77470,23.22188,22.35222,21.96413,26.81268,22.34673,...,18.58534,FBG,52.33333,8,impaired,1062,1648.75000,0,687.63000,109.10000
1062_8_RBG,18.06094,22.27148,23.32116,23.85851,26.82224,23.27269,23.00521,23.22494,26.22146,23.18063,...,16.47109,RBG,256.50000,8,impaired,1062,1648.75000,0,687.63000,109.10000


In [7]:
y_3class = fg['gluc_tol']         # ['normal', 'impaired', 'diabetic']
y_3class_num = y_3class.map({'normal': 0, 'impaired': 1, 'diabetic': 2})  # [0, 1, 2]
y_bg = fg['bg']                            # random/fasted blood glucoose
ogtt_dict = ap['OGTT (AUC)'].to_dict() 
y_ogtt = fg['animal'].map(ogtt_dict)  

In [8]:
# from https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
# Used to find all subsets of a model given its terms 
from itertools import chain, combinations

def powerset(iterable):
    # powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

## Testing more nested models

### Interpretation of p-value:
### if p > 0.05, you should choose the simpler, nested model
### if p < 0.05, you may need to choose the more complex model
#### Therefore, including that term is important for the model's predictive performance


In [38]:
feature = random.sample(list(feat_cols), 1)[0]
print(feature)

nested_model_formulas = []
all_terms = ['ogtt', 'sampling', 'sampling:ogtt']
# all_terms = ['week', 'gluc_tol', 'sampling', 'gluc_tol:sampling', 'gluc_tol:week']
full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 

# for subset in powerset(all_terms):
#     nested_model_formulas.append(' + '.join(subset))

for subset in [x for x in combinations(all_terms, r=len(all_terms)-1)][::-1]:
    formula = ' + '.join(subset)
    print(formula)
    nested_model_formulas.append(formula)

# nested_model_formulas[0] = '1'  # The empty powerset [''] should be specified as intercept-only, so use '1'
# full_model = smf.ols(f'{feature} ~ week + ogtt + sampling + ogtt:sampling + ogtt:week', data=data).fit()
full_model = smf.ols(full_formula, data=data).fit()

# print(dmatrices(full_formula, data=data)[1])

print('p-value, difference in df')

for nested_model_formula in nested_model_formulas:
    formula = f'{feature} ~ {nested_model_formula}'
    nested_model = smf.ols(formula, data=data).fit()
    print([round(x, 4) for x in full_model.compare_lr_test(nested_model)][1:], '\t', formula)

l_131
sampling + sampling:ogtt
ogtt + sampling:ogtt
ogtt + sampling
p-value, difference in df
[nan, 0.0] 	 l_131 ~ sampling + sampling:ogtt
[0.8599, 1.0] 	 l_131 ~ ogtt + sampling:ogtt
[0.0167, 1.0] 	 l_131 ~ ogtt + sampling


## Testing effect of adding week as another covariate to full model 

1. l_515 has super high p-value for week! 
    - so does m_370

In [12]:
data.dtypes[-10:]

l_872       float64
sampling     object
bg          float64
week         object
gluc_tol     object
animal        int64
ogtt        float64
litter       object
insulin     float64
weight      float64
dtype: object

In [18]:
r1.model.exog_names

['Intercept', 'sampling[T.RBG]', 'ogtt:sampling[FBG]', 'ogtt:sampling[RBG]']

In [28]:
smf.ols(f'{feature} ~ gluc_tol + sampling + gluc_tol:sampling', data=data).fit().model.exog_names

['Intercept',
 'gluc_tol[T.impaired]',
 'gluc_tol[T.normal]',
 'sampling[T.RBG]',
 'gluc_tol[T.impaired]:sampling[T.RBG]',
 'gluc_tol[T.normal]:sampling[T.RBG]']

In [23]:
data['ogtt'] = data['ogtt'].astype('float')

In [24]:
feature = random.sample(list(feat_cols), 1)[0]
print(feature)
full = smf.ols(f'{feature} ~ ogtt + sampling + ogtt:sampling', data=data).fit()
r1   = smf.ols(f'{feature} ~        sampling + ogtt:sampling', data=data).fit()
r2   = smf.ols(f'{feature} ~ ogtt            + ogtt:sampling', data=data).fit()
r3   = smf.ols(f'{feature} ~                   ogtt:sampling', data=data).fit()
r4   = smf.ols(f'{feature} ~ ogtt + sampling                ', data=data).fit()


print('p-value,   difference in deg. freedom')

print(full.compare_lr_test(r1)[1:], '\t\t\tOGTT  <-- DOESN\'T COUNT AS A NESTED MODEL (???)')
print(full.compare_lr_test(r2)[1:], '\tSampling')
print(full.compare_lr_test(r3)[1:], '\tOGTT and sampling ')
print(full.compare_lr_test(r4)[1:], '\tOGTT:Sampling interaction')

# with warnings.catch_warnings(record=True) as w:
#     test = smf.mixedlm(f'{feature} ~ gluc_tol + sampling', data, groups=data['animal']).fit(maxiter=200, reml=False)
#     print(test.converged)
#     print(type(test))
# try:
#     print(w[0])
# except:
#     pass

# test.summary()

# test = smf.ols(f'l_555 ~ bg * sampling', data=data).fit()
# display(test.cov_params())
# display(test.conf_int())

l_791
p-value,   difference in deg. freedom
(nan, 0.0) 			OGTT  <-- DOESN'T COUNT AS A NESTED MODEL (???)
(0.015085392438898755, 1.0) 	Sampling
(0.015085392438899201, 1.0) 	OGTT and sampling 
(0.0021422924956412473, 1.0) 	OGTT:Sampling interaction


# Because we only have at most 7 model deg. freedom and 52 residual deg. freedom, what if we just throw every single animal phenotype into the model and see what comes out? 
## Potential phenotypes to use:
1. OGTT AUC
    - Or glucose tolerance (N, I, D)
2. Fasted/fed 
3. Week
4. Insulin AUC
5. Week 12 pre-fast weight
6. Litter

### Then test each of these 

In [31]:
ModelDesc.from_formula('feature ~ OGTT + sampling + sampling:OGTT')

ModelDesc(lhs_termlist=[Term([EvalFactor('feature')])],
          rhs_termlist=[Term([]),
                        Term([EvalFactor('OGTT')]),
                        Term([EvalFactor('sampling')]),
                        Term([EvalFactor('sampling'), EvalFactor('OGTT')])])

In [363]:
feature = random.sample(list(feat_cols), 1)[0]
# feature = 'l_194'
print(feature)
# all_terms = ['ogtt', 'week', 'sampling', 'ogtt:sampling', 'ogtt:week']
all_terms = ['ogtt', 'sampling', 'week', 'weight', 'sampling:ogtt']
# all_terms = ['week', 'gluc_tol', 'sampling', 'gluc_tol:sampling', 'gluc_tol:week']
full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 

# for subset in powerset(all_terms):
#     nested_model_formulas.append(' + '.join(subset))

nested_model_formulas = []
for subset in [x for x in combinations(all_terms, r=len(all_terms)-1)][::-1]:
    formula = ' + '.join(subset)
    print(formula)
    nested_model_formulas.append(formula)

# nested_model_formulas[0] = '1'  # The empty powerset [''] should be specified as intercept-only, so use '1'
# full_model = smf.ols(f'{feature} ~ week + ogtt + sampling + ogtt:sampling + ogtt:week', data=data).fit()
full_model = smf.ols(full_formula, data=data).fit()

# print(dmatrices(full_formula, data=data)[1])

print('p-value, difference in df')

for nested_model_formula, term in zip(nested_model_formulas, all_terms):
    formula = f'{feature} ~ {nested_model_formula}'
    nested_model = smf.ols(formula, data=data).fit()
    print([round(x, 4) for x in full_model.compare_lr_test(nested_model)][1:], 
          '\t', term, 
#           '\t', formula
    )

l_330
sampling + week + weight + sampling:ogtt
ogtt + week + weight + sampling:ogtt
ogtt + sampling + weight + sampling:ogtt
ogtt + sampling + week + sampling:ogtt
ogtt + sampling + week + weight
p-value, difference in df
[nan, 0.0] 	 ogtt
[0.0139, 1.0] 	 sampling
[0.0598, 2.0] 	 week
[0.007, 1.0] 	 weight
[0.0457, 1.0] 	 sampling:ogtt


l_194
sampling + week + insulin + weight + litter + sampling:ogtt + sampling:insulin
ogtt + week + insulin + weight + litter + sampling:ogtt + sampling:insulin
ogtt + sampling + insulin + weight + litter + sampling:ogtt + sampling:insulin
ogtt + sampling + week + weight + litter + sampling:ogtt + sampling:insulin
ogtt + sampling + week + insulin + litter + sampling:ogtt + sampling:insulin
ogtt + sampling + week + insulin + weight + sampling:ogtt + sampling:insulin
ogtt + sampling + week + insulin + weight + litter + sampling:insulin
ogtt + sampling + week + insulin + weight + litter + sampling:ogtt
p-value, difference in df
[nan, 0.0] 	 ogtt
[0.1714, 1.0] 	 sampling
[0.0131, 2.0] 	 week
[nan, 0.0] 	 insulin
[0.2889, 1.0] 	 weight
[0.949, 3.0] 	 litter
[0.1021, 1.0] 	 sampling:ogtt
[0.384, 1.0] 	 sampling:insulin


# Using `statsmodels.ap.stats.anova_lm()` function for likelihood ratio test of nested models


# Should I use Type 1, 2, or 3 ANOVA? 
https://towardsdatascience.com/anovas-three-types-of-estimating-sums-of-squares-don-t-make-the-wrong-choice-91107c77a27a 
https://stats.stackexchange.com/questions/20452/how-to-interpret-type-i-type-ii-and-type-iii-anova-and-manova
### The stackexchange answer says:
1. Order of specifying variables in model matters with type 1 when you have unbalanced data
2. type 2 is almost never used in practice 
2. Type 3 is conservative and required by the FDA, but statisticians find this approach "egregious" 
### R uses type 1 by default, and I can't confirm that type 3 fits my research question, so I will use type 1

In [223]:
feature = random.sample(list(feat_cols), 1)[0]

## Using OGTT

In [224]:
# all_terms = ['ogtt', 'sampling', 'week', 'weight', 'insulin', 'sampling:ogtt', 'insulin:ogtt']
all_terms = ['ogtt', 'sampling', 'sampling:ogtt']
# all_terms = ['week', 'gluc_tol', 'sampling', 'gluc_tol:sampling', 'gluc_tol:week']
full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
full_model = smf.ols(full_formula, data=data).fit()
t1 = sm.stats.anova_lm(full_model, typ=1)
t2 = sm.stats.anova_lm(full_model, typ=2)
t3 = sm.stats.anova_lm(full_model, typ=3)
pd.concat([t1, t2, t3], axis=1).filter(regex='PR')

,PR(>F),PR(>F),PR(>F)
sampling,0.13234,0.13234,0.01398
ogtt,0.05936,0.05936,0.41888
sampling:ogtt,0.00324,0.00324,0.00324
Residual,NaN,NaN,NaN
Intercept,NaN,NaN,0.00000


# Using glucose tolerance (N, I, D) which is unbalanced 

In [191]:
# all_terms = ['ogtt', 'sampling', 'week', 'weight', 'insulin', 'sampling:ogtt', 'insulin:ogtt']
all_terms = ['gluc_tol', 'sampling', 'sampling:gluc_tol']
# all_terms = ['week', 'gluc_tol', 'sampling', 'gluc_tol:sampling', 'gluc_tol:week']
full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
full_model = smf.ols(full_formula, data=data).fit()
t1 = sm.stats.anova_lm(full_model, typ=1)
t2 = sm.stats.anova_lm(full_model, typ=2)
t3 = sm.stats.anova_lm(full_model, typ=3)
pd.concat([t1, t2, t3], axis=1).filter(regex='PR')

,PR(>F),PR(>F),PR(>F)
gluc_tol,0.00303,0.00303,0.75201
sampling,0.00000,0.00000,0.00000
sampling:gluc_tol,0.01275,0.01275,0.01275
Residual,NaN,NaN,NaN
Intercept,NaN,NaN,0.00000


# Throwing the whole enchilada at it (every single piece of rat metadata) 

In [497]:
feature = random.sample(list(feat_cols), 1)[0]
print(feature)
all_terms = ['ogtt', 'sampling',  'sampling:ogtt', 'insulin', 'week', 'weight', 'litter']

full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
full_model = smf.ols(full_formula, data=data).fit()

t1 = sm.stats.anova_lm(full_model, typ=1)
t2 = sm.stats.anova_lm(full_model, typ=2)
t3 = sm.stats.anova_lm(full_model, typ=3)
# pd.concat([t1, t2, t3], axis=1).filter(regex='PR').loc[['sampling', 'ogtt', 'sampling:ogtt', 'gluc_tol',
#                                                        'sampling:gluc_tol', 'insulin', 'sampling:insulin']]
display(t1[['PR(>F)']])

m_355


,PR(>F)
sampling,0.00051
week,0.62324
litter,0.00144
ogtt,0.00141
sampling:ogtt,0.26872
insulin,0.43712
weight,0.85386
Residual,NaN


In [150]:
import seaborn as sns


In [139]:
pd.set_option('display.float_format', '{:.5f}'.format)

In [129]:
all_terms = ['ogtt', 'sampling', 'sampling:ogtt', 'week', 'weight', 'insulin', 'insulin:ogtt']

full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
full_model = smf.ols(full_formula, data=data).fit()

sm.stats.anova_lm(full_model, typ=3)

,sum_sq,df,F,PR(>F)
Intercept,77.1948,1.0,197.7096,2.3617e-19
sampling,2.5457,1.0,6.5200,1.3639e-02
ogtt,0.1219,1.0,0.3121,5.7879e-01
sampling:ogtt,2.5396,1.0,6.5045,1.3745e-02
week,0.6258,1.0,1.6029,2.1114e-01
weight,0.4798,1.0,1.2290,2.7271e-01
insulin,0.0211,1.0,0.0540,8.1723e-01
insulin:ogtt,0.0332,1.0,0.0851,7.7171e-01
Residual,20.3032,52.0,NaN,NaN


In [76]:
all_terms = ['ogtt', 'sampling', 'sampling:ogtt', 'week']

full_formula = f'{feature} ~ ' + ' + '.join(all_terms) 
full_model = smf.ols(full_formula, data=data).fit()

sm.stats.anova_lm(full_model, typ=3)

,df,sum_sq,mean_sq,F,PR(>F)
sampling,1.0,1.440426,1.440426,0.624133,0.432909
ogtt,1.0,20.469261,20.469261,8.869275,0.004308
sampling:ogtt,1.0,18.491405,18.491405,8.012275,0.006476
week,1.0,0.068572,0.068572,0.029712,0.863778
Residual,55.0,126.933647,2.307884,NaN,NaN
